#CNN Binary search using pytorch

In [3]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,random_split
from torchvision import datasets

In [4]:
transform=transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [5]:
dataset=datasets.ImageFolder(root="D:\\testing",transform=transform)

In [6]:
print(len(dataset))
print(dataset.classes)
print(dataset[0])

46
['car', 'nature']
(tensor([[[-0.4039, -0.3961, -0.3961,  ..., -0.5059, -0.5059, -0.5137],
         [-0.3961, -0.4039, -0.4118,  ..., -0.4980, -0.4980, -0.5059],
         [-0.3882, -0.4039, -0.4196,  ..., -0.4667, -0.4667, -0.4745],
         ...,
         [ 0.2078,  0.3176,  0.4431,  ..., -0.6627, -0.6157, -0.5765],
         [ 0.5137,  0.4588,  0.3333,  ..., -0.6784, -0.6314, -0.5922],
         [ 0.1059,  0.0667, -0.0118,  ..., -0.6706, -0.6314, -0.5922]],

        [[-0.2627, -0.2549, -0.2549,  ..., -0.3490, -0.3490, -0.3569],
         [-0.2549, -0.2627, -0.2706,  ..., -0.3412, -0.3412, -0.3490],
         [-0.2627, -0.2784, -0.2941,  ..., -0.3255, -0.3255, -0.3333],
         ...,
         [ 0.2078,  0.3176,  0.4431,  ..., -0.7569, -0.7098, -0.6706],
         [ 0.5137,  0.4588,  0.3333,  ..., -0.7725, -0.7255, -0.6863],
         [ 0.1059,  0.0667, -0.0118,  ..., -0.7647, -0.7255, -0.6863]],

        [[-0.0902, -0.0824, -0.0824,  ..., -0.1608, -0.1608, -0.1686],
         [-0.0824, -0.0

In [7]:
train_size=int(0.8*len(dataset))

In [8]:
train_size

36

In [9]:
test_size=len(dataset)-train_size

In [10]:
test_size

10

In [11]:
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])

In [12]:
len(train_dataset)

36

In [13]:
len(test_dataset)

10

In [14]:
train_loader=DataLoader(train_dataset,batch_size=10,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=4,shuffle=False)

In [15]:
import torch.nn as nn

In [16]:
class CNNBinary(nn.Module):
    def __init__(self):
        super().__init__()
        #iput shape (10,3,224,224)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=10,kernel_size=3,stride=1,padding=1)
        self.batch1=nn.BatchNorm2d(num_features=10)
        self.relu1=nn.ReLU()
        self.pool1=nn.MaxPool2d(kernel_size=2,stride=2)

        #iput shape (10,10,112,112)
        self.conv2=nn.Conv2d(in_channels=10,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.batch2=nn.BatchNorm2d(num_features=20)
        self.relu2=nn.ReLU()
        self.pool2=nn.MaxPool2d(kernel_size=2,stride=2)

        #input shape (10,20,56,56)
        self.linear1=nn.Linear(20*56*56,120)
        self.relu3=nn.ReLU()
        self.linear2=nn.Linear(120,60)
        self.relu4=nn.ReLU()
        self.linear3=nn.Linear(60,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,data):
        out=self.conv1(data)
        out=self.batch1(out)
        out=self.relu1(out)
        out=self.pool1(out)
        out=self.conv2(out)
        out=self.batch2(out)
        out=self.relu2(out)
        out=self.pool2(out)

        out=out.view(-1,20*56*56)#Flatten

        out=self.linear1(out)
        out=self.relu3(out)
        out=self.linear2(out)
        out=self.relu4(out)
        out=self.linear3(out)
        out=self.sigmoid(out)

        return out 

In [17]:
model=CNNBinary()

In [18]:
learning_rate=0.01

In [19]:
criterion=nn.BCELoss()

In [20]:
import torch.optim as optim

In [21]:
optimizer=optim.Adam(model.parameters(),learning_rate)

In [22]:
def training(model,train_loader,criterion,optimizer,epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss=0
        for inputs,labels in train_loader:
            labels=labels.unsqueeze(1).float()
            #forward and output
            outputs=model(inputs)

            optimizer.zero_grad()

            #loss calcute
            loss=criterion(outputs,labels)

            #backward propagation and weight updation
            loss.backward()

            optimizer.step()

            running_loss+=loss.item()

        print(f"the epochs {epoch+1}/{epochs} and loss is {running_loss/len(train_loader)}")
        

In [23]:
training(model,train_loader,criterion,optimizer,50)

the epochs 1/50 and loss is 41.833504885435104
the epochs 2/50 and loss is 50.83333396911621
the epochs 3/50 and loss is 47.5
the epochs 4/50 and loss is 42.5
the epochs 5/50 and loss is 45.833333015441895
the epochs 6/50 and loss is 45.833333015441895
the epochs 7/50 and loss is 45.833333015441895
the epochs 8/50 and loss is 49.16666603088379
the epochs 9/50 and loss is 45.833333015441895
the epochs 10/50 and loss is 49.16666603088379
the epochs 11/50 and loss is 47.5
the epochs 12/50 and loss is 45.833333015441895
the epochs 13/50 and loss is 44.16666650772095
the epochs 14/50 and loss is 45.833333015441895
the epochs 15/50 and loss is 47.5
the epochs 16/50 and loss is 49.16666603088379
the epochs 17/50 and loss is 44.16666650772095
the epochs 18/50 and loss is 50.83333396911621
the epochs 19/50 and loss is 49.16666603088379
the epochs 20/50 and loss is 47.5
the epochs 21/50 and loss is 45.833333015441895
the epochs 22/50 and loss is 47.5
the epochs 23/50 and loss is 49.1666660308837

In [24]:
#testing
total=0
correct=0
with torch.no_grad():
    for inputs,labels in test_loader:
        labels=labels.unsqueeze(1).float()

        pred=model(inputs)

        total+=labels.size(0)
        predict=(pred>0.5).float()
        correct+=(predict==labels).sum().item()
    accuracy=correct/total
    print(f"the total accuracy is {accuracy}")



the total accuracy is 0.5
